In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pylsd.lsd import lsd
%matplotlib inline

In [2]:
img_path = "/Users/Chihiro/Personal/10_Projects/GPC/get_plant_plate_txt/image/20200411_1.jpg"
output_dir = "/Users/Chihiro/Desktop/"

In [77]:
def get_lines_hough(img, img_gray):
    #     ハフ変換
    lines = cv2.HoughLines(img_gray, 
                            rho=1, 
                            theta=np.pi/360, 
                            threshold=1000)
    print(len(lines))
    
    # 検出した線の描画
    for line in lines:
        for rho, theta in line:
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a*rho
            y0 = b*rho
            x1 = int(x0 + 1000*(-b))
            y1 = int(y0 + 1000*(a))
            x2 = int(x0 - 1000*(-b))
            y2 = int(y0 - 1000*(a))

            cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 10)
    cv2.imwrite(output_dir + "lines_hough.png", img)

In [88]:
def get_lines_hough_p(img, img_gray):
    img_g = cv2.GaussianBlur(img_gray,(5,5),1)
    lower = 10
    upper = 250
    img_g = cv2.Canny(img_g,lower,upper,apertureSize = 3)
    linesH = cv2.HoughLinesP(img_g, rho=1, theta=np.pi/360, threshold=30, minLineLength=50, maxLineGap=5)
    
    print(len(linesH))
    for line in linesH:
        x1, y1, x2, y2 = line[0]

        # 赤線を引く
        img = cv2.line(img, (x1,y1), (x2,y2), (0,0,255), 10)
    cv2.imwrite(output_dir + "lines_houghp.png", img) 

In [5]:
def get_lines_LSD(img, img_gray):
    lines = lsd(img_gray)
    for line in lines:
        x1, y1, x2, y2 = map(int,line[:4])
        img = cv2.line(img, (x1,y1), (x2,y2), (0,0,255), 3)
        if (x2-x1)**2 + (y2-y1)**2 > 1000:
           # 赤線を引く
           img = cv2.line(img, (x1,y1), (x2,y2), (0,0,255), 3)
            
    cv2.imwrite(output_dir + "lines_LSD.png", img) 

In [84]:
def get_lines(base_img, img):
#     img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray = img
#     直線検出（ハフ変換)
#     get_lines_hough(base_img, img_gray)
    
# #     直線検出（確率的ハフ変換）
    get_lines_hough_p(base_img, img_gray)

# #     直線検出（LSD）
#     get_lines_LSD(img, img_gray)

In [51]:
def mask_img(img):
#     RGBの比率とGの量を元にマスクできないか試す。
#     RGBの各値が70未満の値は0にする
#         ⇨プレートだけ、目立つようにはできなかった。
#     img = np.where(img >= 60, 255,0).astype('uint8')

#     上から下に一行ずつループ
    for i_row, row in enumerate(img):
        print(i_row)
        for i_column, column in enumerate(row):
            B, G, R = column[0], column[1], column[2]
            if (91 >= B) and (B >= 71):
                if  (92>=G) and  (G >= 62):
                    if (32 >= R) and (R >= 12):
                        img[i_row][i_column] = np.array([0, 0, 0])
                    else:
                         img[i_row][i_column] = np.array([255, 255, 255])
                else:
                     img[i_row][i_column] = np.array([255, 255, 255])
            else:
                img[i_row][i_column] = np.array([255, 255, 255])


#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     plt.imshow(img)
#     plt.show()
    return img

In [54]:
def binalize(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     img_gray = cv2.GaussianBlur(img_gray,(5,5),0)
        
    #  二値化 
    retval, img_bin = cv2.threshold(img_gray,220,255,cv2.THRESH_BINARY)
    
    # 大津の二値化 
#     retval, img_bin = cv2.threshold(img_gray, 0, 255, cv2.THRESH_OTSU)
    
    #出力
    cv2.imwrite(output_dir + "binaly_bin_220.png", img_bin)
    return img_bin

In [89]:
base_img = cv2.imread(img_path)
img = base_img
# img_masked = mask_img(img)
img_bin = binalize(edited_img)
get_lines(base_img, img_bin)

212
